# 3D Droplet Oscillation

Results published: hopefully at some point!

It is part of the BoSSS-long-term validation test suite, which consists of 
several computationally expensive test-cases (runtime in the order of days),
which are performed on a regular basis in order to validate the 
physical correctness of BoSSS simulations.


### Preliminaries

This example can be found in the source code repository as as `Droplet3D.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [ ]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Databases loaded: 
Capacity: 0
Count: 0



Error: System.ApplicationException: Already called.
   at BoSSS.Application.BoSSSpad.BoSSSshell.InitTraceFile() in D:\BoSSS-experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 170
   at BoSSS.Application.BoSSSpad.BoSSSshell.Init() in D:\BoSSS-experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 97
   at Submission#70.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

## Initialization tasks

Loading the `XNSE_Solver` and additional namespace:

In [ ]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater;
using NUnit.Framework;
using BoSSS.Application.XNSE_Solver.Logging;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.Timestepping;

Initialization of the Workflow management; there `OscillatingDroplet3D` is the project name which is used name all computations (aka. sessions):

In [ ]:
BoSSSshell.WorkflowMgm.Init("OscillatingDroplet3D");

Project name is set to 'OscillatingDroplet3D'.
Opening existing database 'D:\local\OscillatingDroplet3D'.


Overview on the available *Execution Queues* (aka. *Batch Processors*, aka. *Batch System*); these e.g. Linux HPC clusters on which compute jobs can be executed.

In [ ]:
ExecutionQueues

index,type,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode
0,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,D:\local\binaries,False,LocalPC,dotnet,<null>,[ D:\local\ == ],,,,,
1,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\hpccluster\hpccluster-scratch\smuda\binaries,False,FDY-WindowsHPC,dotnet,,[ \\hpccluster\hpccluster-scratch\smuda\ == ],FDY\smuda,DC2,<null>,Normal,True


For this example (which is part of the BoSSS validation tests), a *default queue* is selected to run all jobs in the convergence study:

In [ ]:
var myBatch = ExecutionQueues[1];
//var myBatch = GetDefaultQueue();
myBatch

DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,AllowedDatabasesPaths
\\hpccluster\hpccluster-scratch\smuda\binaries,False,FDY-WindowsHPC,dotnet,FDY\smuda,DC2,<null>,Normal,True,[ \\hpccluster\hpccluster-scratch\smuda\ == ]


In [ ]:
bool defaultQueue = false;
var myDB = OpenOrCreateDatabase(@"\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D");

Opening existing database '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D'.


A switch for running the worksheet locally (since we have a folder structure for the data, which is not copied on Jenkins for testing)

In [ ]:
bool NunitTest = false;

In [ ]:
//foreach(var s in wmg.Sessions)
//    s.Delete(true);

## Verification of Initial Value data

Initial values and parameters for the simulation (intial droplet shape, Ohnesorg number, initial velocity)
were specified by project partner (TU Graz, Group Prof. Brenn).
Details can be found in Document `setup.pdf`, to be found in the same directory as this worksheet.

First, it is verified that the initial values chosen here actually match the specification.

In [ ]:
//MultidimensionalArray[] ReferenceData = new MultidimensionalArray[5];
int[] modes = new int[] { 3 };
List<string> cases = new List<string>();
int nCase = 0;
// add simulations for mode 2
if(modes.Contains(2)) {
    cases.Add("m2_Oh01_eta04");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
    cases.Add("m2_Oh01_eta02");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
    cases.Add("m2_Oh01_eta01");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
}
// add simulations for mode 3
if(modes.Contains(3)) {
    cases.Add("m3_Oh01_eta04");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
    // cases.Add("m3_Oh01_eta03");
    // Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    // nCase++;
    // cases.Add("m3_Oh01_eta015");
    // Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    // nCase++;
}
// add simulations for mode 4
if(modes.Contains(4)) {
    cases.Add("m4_Oh01_eta04");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
    cases.Add("m4_Oh01_eta01");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
    cases.Add("m4_Oh056_eta005");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
}
int numCases = cases.Count;
numCases

case 1: m3_Oh01_eta04


1

Load reference data for all cases; These files contain two columns, i.e. the azimuth angle and the respective droplet radius.

In [ ]:
MultidimensionalArray[] ReferenceData = new MultidimensionalArray[numCases];
for(int iCase = 0; iCase < numCases; iCase++) {
    if(!NunitTest)  
        ReferenceData[iCase] = IMatrixExtensions.LoadFromTextFile($"data/InitialValues/{cases[iCase].Substring(0,2)}/surfaceDrop_{cases[iCase]}.txt");
    else
        ReferenceData[iCase] = IMatrixExtensions.LoadFromTextFile($"surfaceDrop_{cases[iCase]}.txt");
}

Analytical expressions for the reference data (see `setup.pdf`); this is to verify that the definition of Legendre
Functions resp. Polynomials in BoSSS actually matches the definition used by the TU Graz group.

In [ ]:
Dictionary<string, Func<double, double>> casesRadius = new Dictionary<string, Func<double, double>>();

In [ ]:
double caseR_m2eta04(double angle) {
   double radius = 0.966781 + 0.4*SphericalHarmonics.MyLegendre(2,0,Math.Cos(angle));
   return radius; 
}
casesRadius.Add("m2_Oh01_eta04", caseR_m2eta04);

In [ ]:
double caseR_m2eta02(double angle) {
   double radius = 0.991848 + 0.2*SphericalHarmonics.MyLegendre(2,0,Math.Cos(angle));
   return radius; 
}
casesRadius.Add("m2_Oh01_eta02", caseR_m2eta02);

In [ ]:
double caseR_m2eta01(double angle) {
   double radius = 0.997981 + 0.1*SphericalHarmonics.MyLegendre(2,0,Math.Cos(angle));
   return radius; 
}
casesRadius.Add("m2_Oh01_eta01", caseR_m2eta01);

In [ ]:
double caseR_m3eta04(double angle) {
    double radius = 0.977143 - 0.00598442*Math.Cos(angle) + 0.4*SphericalHarmonics.MyLegendre(3,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m3_Oh01_eta04", caseR_m3eta04);

In [ ]:
double caseR_m3eta03(double angle) {
    double radius = 0.987143 - 0.00252468*Math.Cos(angle) + 0.3*SphericalHarmonics.MyLegendre(3,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m3_Oh01_eta03", caseR_m3eta03);

In [ ]:
double caseR_m3eta015(double angle) {
    double radius = 0.996786 - 0.000315584*Math.Cos(angle) + 0.15*SphericalHarmonics.MyLegendre(3,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m3_Oh01_eta015", caseR_m3eta015);

In [ ]:
double caseR_m4eta04(double angle) {
    double radius = 0.981839 + 0.4*SphericalHarmonics.MyLegendre(4,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m4_Oh01_eta04", caseR_m4eta04);

In [ ]:
double caseR_m4eta01(double angle) {
    double radius = 0.998883 + 0.1*SphericalHarmonics.MyLegendre(4,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m4_Oh01_eta01", caseR_m4eta01);

In [ ]:
double caseR_m4eta005(double angle) {
    double radius = 0.999721 + 0.05*SphericalHarmonics.MyLegendre(4,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m4_Oh056_eta005", caseR_m4eta005);

Conversion to cartesian coordinates in order to match the data and verification against analytical expression:

In [ ]:
double[][] refX = new double[numCases][];
double[][] refZ = new double[numCases][];
double[][] caseX = new double[numCases][];
double[][] caseZ = new double[numCases][];
for(int iCase = 0; iCase < numCases; iCase++) {
// int testcase = 4;
// for(int iCase = testcase; iCase < testcase+1; iCase++) {
    double[] angle = ReferenceData[iCase].GetColumn(0);
    double[] radius = ReferenceData[iCase].GetColumn(1);
       
    double RadiusErrorNorm = 0.0;
    int I = angle.Length;
    double[] x1 = new double[I], z1 = new double[I];
    double[] cx1 = new double[I], cz1 = new double[I];
    for(int i = 0; i < I; i++) {
        x1[i] = Math.Sin(angle[i])*radius[i];
        z1[i] = Math.Cos(angle[i])*radius[i];
        
        double radius_expr = casesRadius[cases[iCase]](angle[i]);
        RadiusErrorNorm += (radius[i] - radius_expr).Pow2();     
        
        cx1[i] = Math.Sin(angle[i])*radius_expr;
        cz1[i] = Math.Cos(angle[i])*radius_expr;
    } 
    refX[iCase] = x1;
    refZ[iCase] = z1;
    caseX[iCase] = cx1;
    caseZ[iCase] = cz1;

    RadiusErrorNorm = RadiusErrorNorm.Sqrt();
    Console.WriteLine($"Comparison error for radius in case {iCase + 1}: {RadiusErrorNorm}");
    // Note: since the factors in `setup.pdf` are only provided up to 6 digits, an error threshold of 1e-5 seems reasonable.
    //Assert.LessOrEqual(RadiusErrorNorm, 1e-5, "Error in comparing reference data against Legendre polynomials in BoSSS");
}

Comparison error for radius in case 1: 2.540302895441871E-06


### Plot of Reference Data

In [ ]:
int refCase = 0;
Plot(refX[refCase], refZ[refCase], "Ref-Case", ". black",
    caseX[refCase], caseZ[refCase], "Case", ". blue")

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1 
 
 
 
 
 -0.5 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 
 
 
 
 Ref-Case 
 
 
 Ref-Case 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M716.2,34.7 L758.4,34.7 M53.9,44.9 L62.1,44.9 L70.4,45.1 L78.6,45.3 L86.8,45.6 L95.0,45.9
 L103.2,46.4 L111.3,46.9 L119.4,47.5 L127.4,48.2 L135.4,49.0 L143.4,49.8 L151.3,50.8 L159.2,51.8
 L166.9,52.9 L174.6,54.0 L182.3,55.2 L189.9,56.5 L197.3,57.9 L204.7,59.4 L212.0,60.9 L219.3,62.5
 L226.4,64.1 L233.4,65.8 L240.3,67.6 L247.1,69.4 L253.8,71.3 L260.4,73.3 L266.9,75.3 L273.2,77.3
 L279.5,79.4 L285.6,81.6 L291.6,83.8 L297.4,86.1 L303.1,88.4 L308.8,90.7 L314.2,93.1 L319.6,95.5
 L324.8,98.0 L329.9,100.5 L334.8,103.0 L339.6,105.6 L344.3,108.2 L348.8,110.8 L353.2,113.4 L357.5,116.1
 L361.7,118.8 L365.7,121.5 L369.5,124.2 L373.3,126.9 L376.9,129.7 L380.4,132.4 L383.8,135.2 L387.0,137.9
 L390.2,140.7 L393.2,143.5 L396.1,146.3 L398.8,149.0 L401.5,151.8 L404.1,154.6 L406.5,157.3 L408.9,160.1
 L411.1,162.8 L413.3,165.5 L415.3,168.2 L417.3,170.9 L419.2,173.6 L421.0,176.3 L422.7,178.9 L424.4,181.6
 L426.0,184.2 L427.5,186.7 L428.9,189.3 L430.3,191.8 L431.7,194.4 L433.0,196.8 L434.2,199.3 L435.5,201.7
 L436.6,204.1 L437.8,206.5 L438.9,208.9 L440.0,211.2 L441.1,213.5 L442.2,215.7 L443.2,218.0 L444.3,220.1
 L445.3,222.3 L446.4,224.5 L447.4,226.6 L448.5,228.6 L449.6,230.7 L450.7,232.7 L451.8,234.7 L453.0,236.6
 L454.2,238.6 L455.4,240.5 L456.7,242.3 L458.0,244.2 L459.3,246.0 L460.7,247.8 L462.2,249.6 L463.7,251.3
 L465.3,253.0 L466.9,254.7 L468.5,256.4 L470.3,258.1 L472.1,259.7 L474.0,261.3 L475.9,262.9 L477.9,264.5
 L480.0,266.1 L482.1,267.6 L484.4,269.2 L486.6,270.7 L489.0,272.2 L491.5,273.7 L494.0,275.2 L496.6,276.7
 L499.3,278.2 L502.0,279.7 L504.8,281.2 L507.7,282.7 L510.7,284.2 L513.7,285.7 L516.9,287.2 L520.0,288.7
 L523.3,290.2 L526.6,291.7 L530.0,293.2 L533.4,294.7 L536.9,296.2 L540.5,297.8 L544.1,299.3 L547.7,300.9
 L551.4,302.5 L555.2,304.1 L559.0,305.7 L562.8,307.3 L566.6,309.0 L570.5,310.7 L574.5,312.4 L578.4,314.1
 L582.3,315.8 L586.3,317.6 L590.3,319.4 L594.3,321.2 L598.3,323.0 L602.3,324.9 L606.2,326.8 L610.2,328.7
 L614.1,330.6 L618.0,332.6 L621.9,334.6 L625.8,336.6 L629.6,338.6 L633.4,340.7 L637.1,342.8 L640.8,344.9
 L644.4,347.1 L648.0,349.2 L651.4,351.4 L654.9,353.7 L658.2,355.9 L661.4,358.2 L664.6,360.5 L667.7,362.8
 L670.6,365.2 L673.5,367.5 L676.3,369.9 L678.9,372.4 L681.4,374.8 L683.8,377.2 L686.1,379.7 L688.3,382.2
 L690.3,384.7 L692.2,387.2 L693.9,389.7 L695.5,392.3 L697.0,394.8 L698.3,397.4 L699.4,399.9 L700.4,402.5
 L701.2,405.1 L701.9,407.7 L702.4,410.3 L702.7,412.8 L702.9,415.4 L702.9,418.0 L702.7,420.6 L702.3,423.2
 L701.7,425.7 L701.0,428.3 L700.1,430.8 L699.0,433.3 L697.7,435.8 L696.3,438.3 L694.6,440.8 L692.8,443.3
 L690.8,445.7 L688.6,448.1 L686.2,450.5 L683.7,452.9 L680.9,455.2 L678.0,457.5 L674.9,459.8 L671.7,462.0
 L668.2,464.2 L664.6,466.4 L660.8,468.5 L656.8,470.6 L652.7,472.7 L648.4,474.7 L644.0,476.6 L639.3,478.6
 L634.6,480.4 L629.6,482.3 L624.6,484.0 L619.4,485.8 L614.0,487.5 L608.5,489.1 L602.9,490.7 L597.1,492.2
 L591.2,493.6 L585.2,495.1 L579.1,496.4 L572.9,497.7 L566.5,499.0 L560.1,500.1 L553.5,501.3 L546.9,502.3
 L540.1,503.4 L533.3,504.3 L526.5,505.2 L519.5,506.0 L512.5,506.8 L505.4,507.5 L498.2,508.2 L491.0,508.8
 L483.8,509.3 L476.5,509.8 L469.1,510.2 L461.8,510.6 L454.4,510.9 L447.0,511.2 L439.6,511.4 L432.1,511.5
 L424.7,511.6 L417.3,511.6 L409.8,511.6 L402.4,511.5 L395.0,511.4 L387.6,511.3 L380.2,511.0 L372.9,510.8
 L365.6,510.5 L358.3,510.1 L351.0,509.7 L343.8,509.3 L336.7,508.8 L329.6,508.3 L322.5,507.8 L315.6,507.2
 L308.6,506.6 L301.8,505.9 L295.0,505.2 L288.3,504.5 L281.6,50

### Matching of the Spherical Harmonics against the provided Data

In [ ]:
Dictionary<string, Formula> casesPhi = new Dictionary<string, Formula>();

In [ ]:
var Phi_m2eta04_Init = new Formula(
"Phi1",
false,
"using ilPSP.Utils; " + 
"double Phi1(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.966781*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.4*SphericalHarmonics.MyRealSpherical(2, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"}");
casesPhi.Add("m2_Oh01_eta04", Phi_m2eta04_Init);

In [ ]:
var Phi_m2eta02_Init = new Formula(
"Phi4",
false,
"using ilPSP.Utils; " + 
"double Phi4(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.991848*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.2*SphericalHarmonics.MyRealSpherical(2, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m2_Oh01_eta02", Phi_m2eta02_Init);

In [ ]:
var Phi_m2eta01_Init = new Formula(
"Phi4",
false,
"using ilPSP.Utils; " + 
"double Phi4(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.997981*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.1*SphericalHarmonics.MyRealSpherical(2, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m2_Oh01_eta01", Phi_m2eta01_Init);

In [ ]:
var Phi_m3eta04_Init = new Formula(
"Phi2",
false,
"using ilPSP.Utils; " + 
"double Phi2(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    (0.977143 - 0.00598442*Math.Cos(theta))*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.4*SphericalHarmonics.MyRealSpherical(3, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m3_Oh01_eta04", Phi_m3eta04_Init);

In [ ]:
var Phi_m3eta03_Init = new Formula(
"Phi2",
false,
"using ilPSP.Utils; " + 
"double Phi2(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    (0.987143 - 0.00252468*Math.Cos(theta))*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.3*SphericalHarmonics.MyRealSpherical(3, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m3_Oh01_eta03", Phi_m3eta03_Init);

In [ ]:
var Phi_m3eta015_Init = new Formula(
"Phi2",
false,
"using ilPSP.Utils; " + 
"double Phi2(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    (0.996786 - 0.000315584*Math.Cos(theta))*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.15*SphericalHarmonics.MyRealSpherical(3, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m3_Oh01_eta015", Phi_m3eta015_Init);

In [ ]:
var Phi_m4eta04_Init = new Formula(
"Phi3",
false,
"using ilPSP.Utils; " + 
"double Phi3(double[] X) { " +    
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.981839*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.4*SphericalHarmonics.MyRealSpherical(4, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m4_Oh01_eta04", Phi_m4eta04_Init);

In [ ]:
var Phi_m4eta01_Init = new Formula(
"Phi3",
false,
"using ilPSP.Utils; " + 
"double Phi3(double[] X) { " +    
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.998883*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.1*SphericalHarmonics.MyRealSpherical(4, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m4_Oh01_eta01", Phi_m4eta01_Init);

In [ ]:
var Phi_m4eta005_Init = new Formula(
"Phi5",
false,
"using ilPSP.Utils; " + 
"double Phi5(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.999721*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.05*SphericalHarmonics.MyRealSpherical(4, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m4_Oh056_eta005", Phi_m4eta005_Init);

In [ ]:
// IBoundaryAndInitialData[] Phi_iCase = new IBoundaryAndInitialData[]  { Phi1Init, Phi2Init, Phi3Init, Phi4Init, Phi5Init};

In [ ]:
for(int iCase = 0; iCase < numCases; iCase++) {
    double[] angle = ReferenceData[iCase].GetColumn(0);
    //double[] xI = refX[iCase];   
    //double[] zI = refZ[iCase];
    int I = angle.Length;
    
    double PhiErr = 0;
    for(int i = 0; i < I; i++) {
        double radius_expr = casesRadius[cases[iCase]](angle[i]);    
        double x1 = Math.Sin(angle[i])*radius_expr;
        double z1 = Math.Cos(angle[i])*radius_expr;
    
        PhiErr += casesPhi[cases[iCase]].Evaluate(new Vector(x1, 0, z1), 0.0).Abs();
    }
    Console.WriteLine($"Phi error for case {iCase}: {PhiErr}");
    Assert.LessOrEqual(PhiErr, 1e-10, "Level-Set function is not zero at desired surface.");
    
    Assert.IsTrue(casesPhi[cases[iCase]].Evaluate(new Vector(1e-5, 1e-5, 1e-5), 0.0) < 0, "Inside must be phase A/negative");
    Assert.IsTrue(casesPhi[cases[iCase]].Evaluate(new Vector(1e+1, 1e+1, 1e+1), 0.0) > 0, "Outside must be phase B/positive");
}

Phi error for case 0: 2.1094237467877974E-14


### Initial Velocities

In [ ]:
//var anaVel = new IBoundaryAndInitialData[numCases,3];
Dictionary<string, (IBoundaryAndInitialData velX, IBoundaryAndInitialData velY, IBoundaryAndInitialData velZ)> analyticVel = new Dictionary<string, (IBoundaryAndInitialData velX, IBoundaryAndInitialData velY, IBoundaryAndInitialData velZ)>();

for(int iCase = 0; iCase < numCases; iCase++) {
    MultidimensionalArray polVel;
    MultidimensionalArray radVel;
    if (!NunitTest) {
        polVel = IMatrixExtensions.LoadFromTextFile($"data/InitialValues/{cases[iCase].Substring(0,2)}/polarVel_{cases[iCase]}.txt");
        radVel = IMatrixExtensions.LoadFromTextFile($"data/InitialValues/{cases[iCase].Substring(0,2)}/radialVel_{cases[iCase]}.txt");
    } else {
        polVel = IMatrixExtensions.LoadFromTextFile($"polarVel_{cases[iCase]}.txt");
        radVel = IMatrixExtensions.LoadFromTextFile($"radialVel_{cases[iCase]}.txt");
    }
    Assert.IsTrue(ilPSP.Utils.ArrayTools.ListEquals(polVel.GetColumn(0), radVel.GetColumn(0)));
    Assert.IsTrue(ilPSP.Utils.ArrayTools.ListEquals(polVel.GetColumn(1), radVel.GetColumn(1)));
    
    double[] radiusS = polVel.GetColumn(0);
    double[] anglesS = polVel.GetColumn(1);
    double[] polVelS = polVel.GetColumn(2);
    double[] radVelS = radVel.GetColumn(2);
    
    var velX = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 0 };
    velX.SetData(anglesS, radiusS, polVelS, radVelS);
    var velY = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 1 };
    velY.SetData(anglesS, radiusS, polVelS, radVelS);
    var velZ = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 2 };
    velZ.SetData(anglesS, radiusS, polVelS, radVelS);
    
    // anaVel[iCase, 0] = velX;
    // anaVel[iCase, 1] = velY;
    // anaVel[iCase, 2] = velZ;
    analyticVel.Add(cases[iCase], (velX, velY, velZ));
}

In [ ]:
// var anaVel = new IBoundaryAndInitialData[3,3];

// MultidimensionalArray polVel;
// MultidimensionalArray radVel;
// if (!NunitTest) {
//     polVel = IMatrixExtensions.LoadFromTextFile($"data/InitialValues/polarVelCase1.txt");
//     radVel = IMatrixExtensions.LoadFromTextFile($"data/InitialValues/radialVelCase1.txt");
// } else {
//     polVel = IMatrixExtensions.LoadFromTextFile($"polarVelCase1.txt");
//     radVel = IMatrixExtensions.LoadFromTextFile($"radialVelCase1.txt");
// }
// Assert.IsTrue(ilPSP.Utils.ArrayTools.ListEquals(polVel.GetColumn(0), radVel.GetColumn(0)));
// Assert.IsTrue(ilPSP.Utils.ArrayTools.ListEquals(polVel.GetColumn(1), radVel.GetColumn(1)));

// double[] radiusS = polVel.GetColumn(0);
// double[] anglesS = polVel.GetColumn(1);
// double[] polVelS = polVel.GetColumn(2);
// double[] radVelS = radVel.GetColumn(2);

// polVelS.ScaleV(-1.0);
// var velX = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 0 };
// velX.SetData(anglesS, radiusS, polVelS, radVelS);
// var velY = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 1 };
// velY.SetData(anglesS, radiusS, polVelS, radVelS);
// var velZ = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 2 };
// velZ.SetData(anglesS, radiusS, polVelS, radVelS);

// anaVel[0, 0] = velX;
// anaVel[0, 1] = velY;
// anaVel[0, 2] = velZ;
// polVelS = polVel.GetColumn(2);

// radVelS.ScaleV(-1.0);
// velX = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 0 };
// velX.SetData(anglesS, radiusS, polVelS, radVelS);
// velY = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 1 };
// velY.SetData(anglesS, radiusS, polVelS, radVelS);
// velZ = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 2 };
// velZ.SetData(anglesS, radiusS, polVelS, radVelS);

// anaVel[1, 0] = velX;
// anaVel[1, 1] = velY;
// anaVel[1, 2] = velZ;

// polVelS.ScaleV(-1.0);
// velX = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 0 };
// velX.SetData(anglesS, radiusS, polVelS, radVelS);
// velY = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 1 };
// velY.SetData(anglesS, radiusS, polVelS, radVelS);
// velZ = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 2 };
// velZ.SetData(anglesS, radiusS, polVelS, radVelS);

// anaVel[2, 0] = velX;
// anaVel[2, 1] = velY;
// anaVel[2, 2] = velZ;


## Grid Creation

In [ ]:
//foreach(var g in BoSSSshell.WorkflowMgm.Grids)
//   g.Delete(true);

### Quater-Domain grids
(Symmetry planes at $x = 0$ and $y = 0$)

In [ ]:
Dictionary<string, IGridInfo[]> gridTypes = new Dictionary<string, IGridInfo[]>();

In [ ]:
int[] Resolutions = new int[] { 7 };
IGridInfo[] Grids = new IGridInfo[Resolutions.Length];
double scale = 1.0;
for(int i = 0; i < Resolutions.Length; i++) {
    int Res = Resolutions[i];
    string GridName = $"OscillatingDroplet3D_{Res}x{Res}x{2*Res}_wallBC_quarterDomain";

    IGridInfo cachedGrid = wmg.Grids.FirstOrDefault(grid => grid.Name == GridName);
    //cachedGrid = null;
    if(cachedGrid == null) {
        
        // must create new Grid
        double[] xNodes = GenericBlas.Linspace(0, 3*scale, Res + 1);
        double[] yNodes = xNodes;
        double[] zNodes = GenericBlas.Linspace(-3*scale, 3*scale, Res*2 + 1);
        
        var grd = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
        grd.Name = GridName;
        
        grd.DefineEdgeTags(delegate(Vector X) {
            string ret = null;
            if(X.x.Abs() <= 1e-8 || X.y.Abs() <= 1.0e-8)
                ret = IncompressibleBcType.SlipSymmetry.ToString();
            else
                ret = IncompressibleBcType.Wall.ToString();
            return ret;
        });        
        
        Grids[i] = wmg.SaveGrid(grd);
        
    } else {
        //Console.WriteLine($"type: {cachedGrid.GetType()}, is IGridInfo? {cachedGrid is IGridInfo}");
        Console.WriteLine("Grid already found in database - identifid by name " + GridName);
        Grids[i] = cachedGrid;
    }
    
}
gridTypes.Add("wallBC", Grids);

Loading session 3190027a-c3d9-4869-bf85-ed028cbb97e2 failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D\sessions\3190027a-c3d9-4869-bf85-ed028cbb97e2\Session.info'.'
Grid already found in database - identifid by name OscillatingDroplet3D_7x7x14_wallBC_quarterDomain


In [ ]:
// Resolutions = new int[] { 6 };
// Grids = new IGridInfo[Resolutions.Length];
// scale = 1.0;
// for(int i = 0; i < Resolutions.Length; i++) {
//     int Res = Resolutions[i];
//     string GridName = $"OscillatingDroplet3D_{Res}x{Res}x{2*Res}_pressureOutletBC_quarterDomain";

//     IGridInfo cachedGrid = wmg.Grids.FirstOrDefault(grid => grid.Name == GridName);
//     //cachedGrid = null;
//     if(cachedGrid == null) {
        
//         // must create new Grid
//         double[] xNodes = GenericBlas.Linspace(0, 3*scale, Res + 1);
//         double[] yNodes = xNodes;
//         double[] zNodes = GenericBlas.Linspace(-3*scale, 3*scale, Res*2 + 1);
        
//         var grd = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
//         grd.Name = GridName;
        
//         grd.DefineEdgeTags(delegate(Vector X) {
//             string ret = null;
//             if(X.x.Abs() <= 1e-8 || X.y.Abs() <= 1.0e-8)
//                 ret = IncompressibleBcType.SlipSymmetry.ToString();
//             else
//                 ret = IncompressibleBcType.Pressure_Outlet.ToString();
//             return ret;
//         });        
        
//         Grids[i] = wmg.SaveGrid(grd);
        
//     } else {
//         //Console.WriteLine($"type: {cachedGrid.GetType()}, is IGridInfo? {cachedGrid is IGridInfo}");
//         Console.WriteLine("Grid already found in database - identifid by name " + GridName);
//         Grids[i] = cachedGrid;
//     }
    
// }
// gridTypes.Add("pressureOutletBC", Grids);

In [ ]:
// Resolutions = new int[] { 8 };
// Grids = new IGridInfo[Resolutions.Length];
// scale = 1.0;
// for(int i = 0; i < Resolutions.Length; i++) {
//     int Res = Resolutions[i];
//     string GridName = $"OscillatingDroplet3D_{Res}x{Res}x{2*Res}_pressureOutletBC_quarterDomain";

//     IGridInfo cachedGrid = wmg.Grids.FirstOrDefault(grid => grid.Name == GridName);
//     //cachedGrid = null;
//     if(cachedGrid == null) {
        
//         // must create new Grid
//         double[] xNodes = GenericBlas.Linspace(0, 2*scale, Res + 1);
//         double[] yNodes = xNodes;
//         double[] zNodes = GenericBlas.Linspace(-2*scale, 2*scale, Res*2 + 1);
        
//         var grd = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
//         grd.Name = GridName;
        
//         grd.DefineEdgeTags(delegate(Vector X) {
//             string ret = null;
//             if(X.x.Abs() <= 1e-8 || X.y.Abs() <= 1.0e-8)
//                 ret = IncompressibleBcType.SlipSymmetry.ToString();
//             else
//                 ret = IncompressibleBcType.Pressure_Outlet.ToString();
//             return ret;
//         });        
        
//         Grids[i] = wmg.SaveGrid(grd);
        
//     } else {
//         //Console.WriteLine($"type: {cachedGrid.GetType()}, is IGridInfo? {cachedGrid is IGridInfo}");
//         Console.WriteLine("Grid already found in database - identifid by name " + GridName);
//         Grids[i] = cachedGrid;
//     }
    
// }
// gridTypes.Add("refinedTest", Grids);

In [ ]:
gridTypes

key,value
wallBC,[ { Guid = 69b24076-b719-4a03-a7bf-cf59c80df949; Name = OscillatingDroplet3D_7x7x14_wallBC_quarterDomain; Cell Count = 686; Dim = 3 } ]


### Check center of mass

In [ ]:
using BoSSS.Foundation.Quadrature;

In [ ]:
List<(double volume, MultidimensionalArray center)> initValuesMetrics = new List<(double volume, MultidimensionalArray center)>();
foreach(var myCase in cases) {

    double[] xNodes = GenericBlas.Linspace(0, 3*scale, 7 + 1);
    double[] yNodes = xNodes;
    double[] zNodes = GenericBlas.Linspace(-3*scale, 3*scale, 7*2 + 1);
    
    var grd = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
    GridData grdDat = new GridData(grd); 

    int order = 4;
    Basis basis = new Basis(grdDat, 4);
    SinglePhaseField phi = new SinglePhaseField(basis);
    phi.ProjectField(X => casesPhi[myCase].Evaluate(X, 0.0));

    LevelSet lvlSet = new LevelSet(basis, "levelSet");
    lvlSet.Acc(1.0, phi); 
    LevelSetTracker LsTrk = new LevelSetTracker(grdDat, XQuadFactoryHelper.MomentFittingVariants.Saye,  1, new string[] {"A", "B"}, lvlSet);
    LsTrk.UpdateTracker(0.0);

    var SchemeHelper = LsTrk.GetXDGSpaceMetrics(LsTrk.SpeciesIdS.ToArray(), order, 1).XQuadSchemeHelper;

    // volume of droplet
    double volume = 0.0;
    SpeciesId spcId = LsTrk.SpeciesIdS[0];
    var vqs = SchemeHelper.GetVolumeQuadScheme(spcId);
    CellQuadrature.GetQuadrature(new int[] { 1 }, grdDat,
        vqs.Compile(grdDat, order),
        delegate (int i0, int Length, QuadRule QR, MultidimensionalArray EvalResult) {
            EvalResult.SetAll(1.0);
        },
        delegate (int i0, int Length, MultidimensionalArray ResultsOfIntegration) {
            for (int i = 0; i < Length; i++)
                volume += ResultsOfIntegration[i, 0];
        }
    ).Execute();

    // center of mass/geometric center (for incompressible fluid)
    int D = 3;
    MultidimensionalArray center = MultidimensionalArray.Create(D,1);
    CellQuadrature.GetQuadrature(new int[] { D }, grdDat,
        vqs.Compile(grdDat, order),
        delegate (int i0, int Length, QuadRule QR, MultidimensionalArray EvalResult) {
            NodeSet nodes_global = QR.Nodes.CloneAs();
            for (int i = i0; i < i0 + Length; i++) {
                LsTrk.GridDat.TransformLocal2Global(QR.Nodes, nodes_global, i);
                EvalResult.AccSubArray(1.0, nodes_global, new int[] { i - i0, -1, -1 });
            }
        },
        delegate (int i0, int Length, MultidimensionalArray ResultsOfIntegration) {
            for (int i = 0; i < Length; i++) {
                for (int d = 0; d < D; d++) {
                    center[d, 0] += ResultsOfIntegration[i, d];
                }
            }
        }
    ).Execute();

    center.Scale(1.0 / volume);

    initValuesMetrics.Add((volume, center));
}

In [ ]:
initValuesMetrics.ElementAt(2)

Error: System.ArgumentOutOfRangeException: Index was out of range. Must be non-negative and less than the size of the collection. (Parameter 'index')
   at System.Collections.Generic.List`1.get_Item(Int32 index)
   at System.Linq.Enumerable.ElementAt[TSource](IEnumerable`1 source, Int32 index)
   at Submission#114.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
//var g = (wmg.Grids[0] as GridProxy).RealGrid;

In [ ]:
//(g.iGridData as GridData).GlobalBoundingBox

In [ ]:
//wmg.Sessions[0].Delete(true);

## Setup of control objects for all solver runs

In [ ]:
cases

index,value
0,m3_Oh01_eta04


In [ ]:
Dictionary<string, (double Ohnesorge, int AMRlevel, double dt, int timesteps, int saveperiod)> casesSetUp = new Dictionary<string, (double Ohnesorge, int AMRlevel, double dt, int timesteps, int saveperiod)>();

In [ ]:
casesSetUp.Add("m2_Oh01_eta04", (0.1, 1, 5e-3, 1400, 2));
casesSetUp.Add("m2_Oh01_eta02", (0.1, 1, 5e-3, 1400, 2));
casesSetUp.Add("m2_Oh01_eta01", (0.1, 1, 5e-3, 1400, 2));
casesSetUp.Add("m3_Oh01_eta04", (0.1, 2, 25e-4, 1400, 4));
casesSetUp.Add("m3_Oh01_eta03", (0.1, 2, 25e-4, 1400, 4));
casesSetUp.Add("m3_Oh01_eta015", (0.1, 2, 25e-4, 1400, 4));
casesSetUp.Add("m4_Oh01_eta04", (0.1, 1, 5e-3, 1400, 2));
casesSetUp.Add("m4_Oh01_eta01", (0.1, 2, 25e-4, 1400, 4));
casesSetUp.Add("m4_Oh056_eta005", (0.56, 1, 5e-3, 800, 1));

In [ ]:
string[] gridTypeKeys = new string[] { "wallBC" }; ///{ "wallBC", "pressureOutletBC", "refinedTest" };

In [ ]:
List<XNSE_Control> Controls = new List<XNSE_Control>();
Controls.Clear();
int[] DegreeS = new int[] { 3 };
bool[] useInitial = new bool[] { false };
bool[] useNewton = new bool[] { false };

// string grdKey = gridTypeKeys[1];
// Grids = gridTypes[grdKey];

foreach(bool bInitial in useInitial) {
foreach(bool bNewton in useNewton) {
foreach(int k in DegreeS) {
foreach(string grdKey in gridTypeKeys) {
foreach(var grd in gridTypes[grdKey]) {
int iCase = 0;
foreach(var myCase in cases) {
    long J = grd.NumberOfCells;
    int AMRlvl = casesSetUp[myCase].AMRlevel;
    string JobName = $"OD3D_J{J}k{k}_{grdKey}_amr{AMRlvl}_{myCase}";
    if(bInitial) {
        JobName = JobName + "_thirdOrderInit";
    }
    if(bNewton) {
        JobName = JobName + "_Newton";
    }
    Console.WriteLine($"Case {iCase+1}: " + JobName);
    iCase++;

    var C = new XNSE_Control();
    
    C.SetGrid(grd);
    C.SetDGdegree(k);
    C.SessionName = JobName;

    if(!defaultQueue) {
        C.SetDatabase(myDB);
    }
    
    C.InitialValues.Add("Phi", casesPhi[myCase]);
    
    C.PhysicalParameters.IncludeConvection = true;
    C.PhysicalParameters.rho_A = 1;
    C.PhysicalParameters.rho_B = 0.001;
    C.PhysicalParameters.mu_A = casesSetUp[myCase].Ohnesorge;
    C.PhysicalParameters.mu_B = casesSetUp[myCase].Ohnesorge/1000;
    C.PhysicalParameters.reynolds_B = 0.0;
    C.PhysicalParameters.reynolds_A = 0.0;
    C.PhysicalParameters.Sigma = 1;
    C.PhysicalParameters.pFree = 0.0;
    C.PhysicalParameters.mu_I = 0.0;
    C.PhysicalParameters.lambda_I = 0.0;
    C.PhysicalParameters.lambdaI_tilde = -1.0;
    C.PhysicalParameters.betaS_A = 0.0;
    C.PhysicalParameters.betaS_B = 0.0;
    C.PhysicalParameters.betaL = 0.0;
    C.PhysicalParameters.theta_e = 1.5707963267948966;
    C.PhysicalParameters.sliplength = 0.0;
    C.PhysicalParameters.Material = true;
    C.PhysicalParameters.useArtificialSurfaceForce = false;
    
    C.Option_LevelSetEvolution = BoSSS.Solution.LevelSetTools.LevelSetEvolution.StokesExtension;
    C.AdvancedDiscretizationOptions.SST_isotropicMode = SurfaceStressTensor_IsotropicMode.LaplaceBeltrami_ContactLine;
    C.LSContiProjectionMethod = ContinuityProjectionOption.ConstrainedDG;
    
    C.TimeSteppingScheme = TimeSteppingScheme.BDF3;
    if(bNewton) {
        C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    } else {
        C.NonLinearSolver.SolverCode = NonLinearSolverCode.Picard;
        C.NonLinearSolver.ConvergenceCriterion = 1e-9;
    }
    
    C.NonLinearSolver.MaxSolverIterations = 50;
    C.NonLinearSolver.MinSolverIterations = 3;
    C.Timestepper_BDFinit = TimeStepperInit.SingleInit;
    C.Timestepper_LevelSetHandling = LevelSetHandling.Coupled_Once;
    C.TimesteppingMode = AppControl._TimesteppingMode.Transient;
    C.dtFixed = casesSetUp[myCase].dt;
    C.NoOfTimesteps = casesSetUp[myCase].timesteps;
    
    if(AMRlvl > 0) {
        C.AdaptiveMeshRefinement = true;
        C.activeAMRlevelIndicators.Add(
            new AMRonNarrowband() { maxRefinementLevel = AMRlvl }
        );
    }
    
    if(bInitial) {
        C.AddInitialValue("VelocityX#A", analyticVel[myCase].velX);
        //C.AddInitialValue("VelocityX#B", anaVel[myCase.Case - 1, 0]);
        C.AddInitialValue("VelocityY#A", analyticVel[myCase].velY);
        //C.AddInitialValue("VelocityY#B", anaVel[myCase.Case - 1, 1]);
        C.AddInitialValue("VelocityZ#A", analyticVel[myCase].velZ);
        //C.AddInitialValue("VelocityZ#B", anaVel[myCase.Case - 1, 2]);
    }
    
    C.PostprocessingModules.Add(new SphericalHarmonicsLogging() { MaxL = 8, RotSymmetric = true });
    C.PostprocessingModules.Add(new DropletMetricsLogging() { AxisSymmetric = true });
    C.PostprocessingModules.Add(new EnergyLogging());

    C.saveperiod =  casesSetUp[myCase].saveperiod;
    
    C.TracingNamespaces = "*";
    
    Controls.Add(C);
    
}
}
}
}
}
}

Case 1: OD3D_J686k3_wallBC_amr2_m3_Oh01_eta04


In [ ]:
int NC = Controls.Count;
for(int i = 0; i < NC; i++) {
    for(int j = 0; j < NC; j++) {
        if(i == j)
            Assert.IsTrue(Controls[i].Equals(Controls[j]), "Control is not self-equal for " + i);
        else
            Assert.IsFalse(Controls[i].Equals(Controls[j]), "Different Control are wrongly equal for " + i + " and " + j);
    }
}

## Launch Jobs

In [ ]:
Controls.Select(C => C.SessionName)

index,value
0,OD3D_J686k3_wallBC_amr2_m3_Oh01_eta04


In [ ]:
foreach(var ctrl in Controls) {
    var oneJob              = ctrl.CreateJob();
    oneJob.NumberOfMPIProcs = 1;
    oneJob.Activate(myBatch); 
}

Loading session 3190027a-c3d9-4869-bf85-ed028cbb97e2 failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D\sessions\3190027a-c3d9-4869-bf85-ed028cbb97e2\Session.info'.'
Loading session 3190027a-c3d9-4869-bf85-ed028cbb97e2 failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D\sessions\3190027a-c3d9-4869-bf85-ed028cbb97e2\Session.info'.'
Loading session 3190027a-c3d9-4869-bf85-ed028cbb97e2 failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D\sessions\3190027a-c3d9-4869-bf85-ed028cbb97e2\Session.info'.'
Loading session 3190027a-c3d9-4869-bf85-ed028cbb97e2 failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D\sessions\3190027a-c3d9-4869-bf85-ed028cbb97e2\Session.info'.'
Deploying job OD3D_J686k3_wallBC_amr2_m3_Oh01_eta04 ... 
Deploying executables and additional files ...
Deployment directory: \\hpcclust

In [ ]:
//wmg.AllJobs
//myBatch

In [ ]:
// wait for all jobs to finish (up to 5 days, check every 30 minutes)
//BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(TimeOutSeconds:(3600), PollingIntervallSeconds:(60*30));

In [ ]:
// detect failed Jobs in the job management
/*
var suspects = BoSSSshell.WorkflowMgm.AllJobs.Select(kv => kv.Value)
    .Where(job => job.LatestSession.Tags.Contains(SessionInfo.NOT_TERMINATED_TAG)
                  || job.LatestSession.Tags.Contains(SessionInfo.SOLVER_ERROR)).ToArray();
suspects
*/

In [ ]:
//suspects.Count()

In [ ]:
//NUnit.Framework.Assert.IsTrue(suspects.Count() <= 0, $"{suspects.Count()} Failed Jobs of {BoSSSshell.WorkflowMgm.AllJobs.Count()} in total.");

### Inspect the output of some arbitrary job:

In [ ]:
//BoSSSshell.WorkflowMgm.AllJobs.First().Value.ShowOutput();

In [ ]:
//wmg.Sessions[0].Export().WithSupersampling(2).Do()